In [27]:
from intake import open_catalog
import datetime
import numpy as np
import pandas as pd
import scipy.io
import gcsfs
import xarray as xr
import rclv
from copy import deepcopy
from skimage import measure

# load the time of eddy identification
fn = '/home/jovyan/RCLVs/SCS/Eddies_SCS_structure_olddata.mat'
data = scipy.io.loadmat(fn)

data1 = data['eds_28d_stru']
data2 = data['eds_28d_trk_stru']
data3 = data['t_eddy']

# load ssh eddy information
df = pd.read_pickle('ssh_eddy.pkl')

nyear = len(np.arange(1993, 2016, 1))
date_st = np.zeros((nyear*12,))

i = 0
for year in np.arange(1993, 2016, 1):
    for mon in np.arange(1, 13, 1):
        date_st[i] = datetime.date.toordinal(datetime.date(year, mon, 1))
        i += 1
        
x_left, x_right = 100, 132
y_south, y_north = 0, 30
dx = 32 * (x_right - x_left)
dy = 32 * (y_north - y_south)

cal_num = np.zeros((2394, 1)) # the number of the remain day 
cal_num[:, :] = np.NaN 
# define vars to input data

eddy_data = np.zeros((2394, 14))
eddy_data[:, :] = np.NaN
    
for i in np.arange(0, 2394, 1):
#for i in np.arange(0, 2, 1):

    #intru_data = np.zeros((1, 22))
    #intru_data[:, :] = np.NaN
    
    print('eddy id: ' + str(df.id[i]))
    eddy_data[i, 0] = df.id[i]
    eddy_data[i, 1] = df.duration[i]
    #intru_data[0, 0] = df.id[i]
    #intru_data[0, 1] = df.duration[i]
        
    eddy_time = df.time[i] # lifetime of ssh eddy
    
    date0_str = datetime.date.fromordinal(int(eddy_time[0]-366)).strftime('%Y-%m-%d')
    #print('eddy_t0: ' + str(date0_str))
    
    duration = df.duration[i]
    #print('lifespan: ' + str(duration))
    
    date_diff = (date_st - eddy_time[0] + 366).tolist()    # check the number!!!!!!
    
    var_tmp1 = [num for num in date_diff if num >= 0]
    
    if len(var_tmp1) > 0:
        min_diff = min(var_tmp1)
        #print('min_diff: ' + str(min_diff))

        remain_days = df.duration[i] - min_diff # calculate the days left
        #print('remain: ' + str(remain_days))
        cal_num[i] = int(remain_days / 10) # the output of lagrangian data is 10 days
        
        if cal_num[i] > 18:
            cal_num[i] = 18
            
        start_day = eddy_time[0] - 366 + min_diff
        start_str = datetime.date.fromordinal(int(start_day)).strftime('%Y-%m-%d')
        #print('calculation_t0: ' + str(start_str))

        sd_id = np.argwhere(eddy_time == (start_day+366))
        
        if sd_id.shape[0] > 0:
            
            eddy_data[i, 2] = df.area[i][sd_id]
            #intru_data[0, 2] = df.area[i][sd_id]
            
            
            #boundary = df.boundary[i]
            
            #var_tmp2 = boundary[sd_id][0, 0]
        
            #num2 = var_tmp2[1, :] / (y_north - y_south) * dy  # position of y
            #num1 = (var_tmp2[0, :] - 100) / (x_right - x_left) * dx # position of x
    
            #contour0 = np.zeros((var_tmp2.shape[1], 2))
            #contour0[:, 1] = num1
            #contour0[:, 0] = num2
            
            tar_day = start_day + 366 + 10*int(cal_num[i])
            sd_id2 = np.argmin(np.abs(tar_day - eddy_time + 0.01)) 
            
            amptmp = np.abs(df.amplitude[i][sd_id[0][0]:sd_id2+1])
            
            eddy_data[i, 3] = amptmp.mean()
            eddy_data[i, 4] = (amptmp[-1] - amptmp[0]) / amptmp.mean()
            
            vorttmp = df.vorticity[i][sd_id[0][0]:sd_id2+1]
            
            eddy_data[i, 5] = vorttmp.mean()
            
            radtmp = df.radius[i][sd_id[0][0]:sd_id2+1]
            
            eddy_data[i, 6] = radtmp.mean()
            
            eddy_data[i, 7] = df.speed_x[i]
            eddy_data[i, 8] = df.speed_y[i]
            eddy_data[i, 9] = df.cyc[i]
            
            edsptmp = df.edgespd[i][sd_id[0][0]:sd_id2+1]
            
            eddy_data[i, 10] = edsptmp.mean()
            
            ketmp = df.KE[i][sd_id[0][0]:sd_id2+1]
            
            eddy_data[i, 11] = ketmp.mean()
            
            eddy_data[i, 13] = df.center_traj[i][0][0] # lon0
            eddy_data[i, 14] = df.center_traj[i][0][1] # lat0
            #eddy_data[i, 12] = cal_num
            #for nday in np.arange(int(cal_num[i]+1)):
                
                
                # find the 10th 20th ... day
                # 0.01 is to aviod the double id


                #boun_tmp = boundary[sd_id2]
                #amp_tmp = amp[sd_id2]
                #eddy_data[i, nday+3] = amp_tmp
                #leak_data[0, nday+3] = np.sum(measure.points_in_poly(np.array([pxin1d, pyin1d]).T, np.array([boun_tmp[0, :], boun_tmp[1, :]]).T))
                #intru_data[0, nday+3] = np.sum(measure.points_in_poly(np.array([pxout1d, pyout1d]).T, np.array([boun_tmp[0, :], boun_tmp[1, :]]).T))
                
                #if nday == 18:
                #    break
                    
df = pd.DataFrame(eddy_data)
fn = './eddy_data.pkl'
df.to_pickle(fn)
     

eddy id: 1


IndexError: index 14 is out of bounds for axis 1 with size 14